<a href="https://colab.research.google.com/github/maxiperezc/memoscopio/blob/master/HPC/PerezCoto_Maximiliano_ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# 1 Introducción

El siguiente cuaderno realiza la norma de N de 3 dimensiones vectores en forma secuencial,  utilizando el procesador CPU.

<left>$V_0 = (X_0, Y_0, Z_0)$</left>  
<left>$...$</left>

<left>$V_n = (X_n, Y_n, Z_n)$</left>  

<left>Luego,</left>

<left>$||V_n|| = \sqrt( X^2_n + Y^2_n + Z^2_n)  $</left>  

El objetivo es calcular la fuerza correspondiente a N vectores para ser utilizado en ambientes de desarrollo gráfico o con fines de investigación científica.

# 2 Desarrollo

In [13]:
# --------------------------------------------
#@title 3.1 Parámetro de ejecución: cantidad de vectores de 3 dimensiones { vertical-output: true }

cantidad = 5#@param {type: "number"}

from datetime import datetime
import numpy

try:
  if cantidad < 1:
    raise ValueError("Error: La cantidad de vectores debe ser al menos 1.");

# --------------------------------------------

  tiempo_total = datetime.now()

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  # --------------------------------------------
  # CPU - Defino la memoria de los vectores en cpu.
  x_cpu = numpy.random.randn( cantidad )
  x_cpu = x_cpu.astype( numpy.float32() )

  y_cpu = numpy.random.randn( cantidad )
  y_cpu = y_cpu.astype( numpy.float32() )

  z_cpu = numpy.random.randn( cantidad )
  z_cpu = y_cpu.astype( numpy.float32() )

  v_cpu = numpy.empty_like( x_cpu )

  tiempo_bucle = datetime.now()

  for idx in range( 0, cantidad ):
    v_cpu[idx] = numpy.sqrt(pow(x_cpu[idx], 2) + pow(y_cpu[idx], 2) + pow(z_cpu[idx], 2))

  tiempo_bucle = datetime.now() - tiempo_bucle

  # --------------------------------------------


  # CPU - Informo el resultado.
  print( "------------------------------------")
  print( "X: " )
  print( x_cpu )
  print( "------------------------------------")
  print( "Y: " )
  print( y_cpu )
  print( "------------------------------------")
  print( "Z: " )
  print( z_cpu )
  print( "------------------------------------")
  print( "V: " )
  print( v_cpu )
  print( "------------------------------------")
  print( "\n\n")

  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo bucle: ", tiempo_en_ms( tiempo_bucle ), "[ms]" )

except ValueError as VR:
  print(VR)
except: 
  print("Error no atrapado...")

------------------------------------
X: 
[-1.6390595  -1.1043664   0.8207337  ... -0.0231653   0.78886664
  1.5993896 ]
------------------------------------
Y: 
[-0.00521451 -1.4160372   0.12359434 ...  1.7020512   0.49831384
 -0.7463814 ]
------------------------------------
Z: 
[-0.00521451 -1.4160372   0.12359434 ...  1.7020512   0.49831384
 -0.7463814 ]
------------------------------------
V: 
[1.6390761 2.286908  0.8391394 ... 2.4071753 1.0578015 1.916303 ]
------------------------------------



Tiempo Total:  53.978 [ms]
Tiempo bucle:  51.153 [ms]


---
#4 Tabla de pasos
Tabla de pasos de la ejecución del programa:

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo inicial.
CPU      |  numpy.random.randn( Cantidad ) | Inicializa los vectores A, B y R.
CPU      |  for...                | Realiza la norma de los vectores $(X_0, Y_0, Z_0) ... (X_n, Y_n, Z_n) $ , guardando el resultado en $V_n$. 
CPU      |  datetime.now()        | Toma el tiempo final.
CPU      |  print()               | Informa los resultados.



---
# GPU

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [5]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 20.1MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620899 sha256=8c1324d0f4f79026b87ffda44c6f3bf84e0f657038f6990c9de6291949a6ec94
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=a6efcde099aeeafde6e0448e794ad58fc27c37fd92abcce577ec800302cab201
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


# 3 Desarrollo GPU

In [12]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución: cantidad de vectores { vertical-output: true }

cantidad =   5000#@param {type: "number"}
# --------------------------------------------

from datetime import datetime
import numpy

try:
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule
except:
  print("Error: debe cambiar el entorno a GPU e instalar CUDA en el paso anterior antes de continuar.")

try:
  if cantidad < 1:
    raise ValueError("Error: La cantidad de vectores debe ser al menos 1.");

  tiempo_total = datetime.now()
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  # CPU - Defino la memoria de los vectores en cpu.
  x_cpu = numpy.random.randn( cantidad )
  x_cpu = x_cpu.astype( numpy.float32() )

  y_cpu = numpy.random.randn( cantidad )
  y_cpu = y_cpu.astype( numpy.float32() )

  z_cpu = numpy.random.randn( cantidad )
  z_cpu = y_cpu.astype( numpy.float32() )

  v_cpu = numpy.empty_like( x_cpu )

  # CPU - reservo la memoria GPU.
  x_gpu = cuda.mem_alloc( x_cpu.nbytes )
  y_gpu = cuda.mem_alloc( y_cpu.nbytes )
  z_gpu = cuda.mem_alloc( z_cpu.nbytes )
  v_gpu = cuda.mem_alloc( v_cpu.nbytes )

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod( x_gpu, x_cpu )
  cuda.memcpy_htod( y_gpu, y_cpu )
  cuda.memcpy_htod( z_gpu, z_cpu )
  cuda.memcpy_htod( v_gpu, v_cpu )

  # CPU - Defino la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_norm( int n, float *X, float *Y, float *Z, float *V )
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if( idx < n )
    {
      V[idx] = sqrt(pow(X[idx], 2) + pow(Y[idx], 2) + pow(Z[idx], 2));
    }
  }
  """)
  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel_norm")

  tiempo_gpu = datetime.now()

  dim_hilo = 256
  dim_bloque = numpy.int( (cantidad+dim_hilo-1) / dim_hilo )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  kernel( numpy.int32(cantidad), x_gpu, y_gpu, z_gpu, v_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

  tiempo_gpu = datetime.now() - tiempo_gpu

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( v_cpu, v_gpu )

  # CPU - Informo el resutlado.
  print( "------------------------------------")
  print( "X: " )
  print( x_cpu )
  print( "------------------------------------")
  print( "Y: " )
  print( y_cpu )
  print( "------------------------------------")
  print( "Z: " )
  print( z_cpu )
  print( "------------------------------------")
  print( "V: " )
  print( v_cpu )
  print( "------------------------------------")
  print( "\n\n")

  tiempo_total = datetime.now() - tiempo_total

  print( "Cantidad de elementos: ", cantidad )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

except ValueError as VR:
  print(VR)
except: 
  print("Error no atrapado...")

Thread x:  256 , Bloque x: 20
------------------------------------
X: 
[ 1.3746057  -0.01878837 -1.0449567  ...  0.4825686  -0.37084395
 -0.92687875]
------------------------------------
Y: 
[ 0.61512613  1.1581683   0.03266733 ...  1.5751969   1.5738485
 -1.8131617 ]
------------------------------------
Z: 
[ 0.61512613  1.1581683   0.03266733 ...  1.5751969   1.5738485
 -1.8131617 ]
------------------------------------
V: 
[1.6267456 1.6380051 1.0459775 ... 2.2793338 2.2564404 2.7265756]
------------------------------------



Cantidad de elementos:  5000
Thread x:  256 , Bloque x: 20
Tiempo CPU:  4.102 [ms]
Tiempo GPU:  0.457 [ms]


---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad ) | Inicializa los vectoes A, B y R.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

Si bien el ejercicio realizado no presenta una gran complejidad, creo que fue necesario realizar algo simple para familiarizarme con el lenguaje de Python y como es el funcionamiento de los cuadernos de Colab; tanto para la declaración de funciones, como de variables y excepciones. Tambien me sirvio como puntapie para aprender sobre las distintas estructuras y funciones que ofrece Numpy[6]

En cuanto a los resultado, si bien el calculo realizado no requiere de mucho calculo por parte de la CPU, al comparar los valores entre la ejecición en serie y la ejecución en paralelo se ve una "ENORME" diferencia.

En promedio, con la ejecución en serie obtuve un resultado de 63.221 [ms] (tamando 10 ejecuciones con 10000 objetos). Por otra parte, con la ejecución en paralelo, el calculo de las diferentes energias se resolvió en un promedio de 0.089 [ms].

### Pasos mas relevantes

1- Reservar memoria en GPU (cuda.mem_alloc).

2- Copiar datos en memoria GPU (cuda.memcpy_htod).

3- Definir la función que va a ejecutar el Kernel.

4- Calcular las dimensiones.


---

# 6 Bibliografia

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[6] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)


